In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ..

/home/ubuntu/sustaingym


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from sustaingym.envs.evcharging import EVChargingEnv, RealTraceGenerator, \
    GMMsTraceGenerator, DiscreteActionWrapper, MultiAgentEVChargingEnv
from sustaingym.algorithms.evcharging.baselines import \
    RLLibAlgorithm, GreedyAlgorithm, RandomAlgorithm, MPC, OfflineOptimal
from sustaingym.envs.evcharging.event_generation import AbstractTraceGenerator
from sustaingym.envs.evcharging.utils import DATE_FORMAT, DEFAULT_PERIOD_TO_RANGE, SiteStr

day = '2021-07-06'
dg = RealTraceGenerator('caltech', (day, day))
# '2021-07-05', '2021-07-18'
env = EVChargingEnv(dg, vectorize_obs=False)

In [3]:
ga = GreedyAlgorithm(env)
ra = RandomAlgorithm(env)
mpc = MPC(env)
oo = OfflineOptimal(env)

name2algo = {
    'ga': ga,
    'ra': ra,
    'mpc': mpc,
    'oo': oo
}

name2action = {}
name2res = {}

for name in name2algo:
    print(name)
    res, action = name2algo[name].run([1], return_pilot_signals=True)
    name2action[name] = action
    name2res[name] = res

ga
[1]


100%|██████████| 1/1 [00:07<00:00,  7.23s/it]


ra
[1]


100%|██████████| 1/1 [00:07<00:00,  7.25s/it]


mpc
[1]


100%|██████████| 1/1 [00:14<00:00, 14.13s/it]


oo
[1]


  0%|          | 0/1 [00:00<?, ?it/s]

: 

: 

In [ ]:
plt.title('Sum of pilot signals on timestep')
plt.xlabel('timestep')
plt.ylabel('Pilot signal (A?)')
for name in name2algo:
    action = name2action[name].sum(axis=1)
    plt.plot(action, label=name)
plt.legend()
plt.show()

In [2]:
from sustaingym.scripts.evcharging.train_rllib import get_algo

from ray import rllib

config = {
    "algo": "a2c",
    "dp": "Summer 2021",  # train date period
    "site": 'caltech',
    "discrete": False,
    "multiagent": False,
    "periods_delay": 0,
    "seed": 123
}

_, algo = get_algo(config)
algo.restore("~/ray_results/A2C_evcharging_2023-04-17_17-13-21ow8xsowy/checkpoint_000025")

2023-05-12 18:06:33,529	INFO worker.py:1625 -- Started a local Ray instance.
(RolloutWorker pid=698560) 2023-05-12 18:06:42,541	WARNING env.py:155 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
2023-05-12 18:06:42,813	INFO trainable.py:172 -- Trainable.setup took 11.236 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


ValueError: Could not recover from checkpoint as it does not exist on local disk and was not available on cloud storage or another Ray node. Got checkpoint path: ~/ray_results/A2C_evcharging_2023-04-17_17-13-21ow8xsowy/checkpoint_000025 and IP None